## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-26-22-52-34 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/01/26/us/min...
1,2026-01-26-22-52-00 +0000,wsj,Democrats want changes to the Department of Ho...,https://www.wsj.com/politics/policy/government...
2,2026-01-26-22-48-38 +0000,nyt,Killings in Minneapolis Invert Usual Dynamic O...,https://www.nytimes.com/2026/01/26/us/politics...
3,2026-01-26-22-48-13 +0000,nypost,NYPD cop who shot ‘vicious’ raccoon on NYC boa...,https://nypost.com/2026/01/26/us-news/nypd-cop...
4,2026-01-26-22-48-07 +0000,nypost,"Shackled, greasy-haired, disgraced Olympian Ry...",https://nypost.com/2026/01/26/us-news/former-o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2423/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
54,trump,46
10,minneapolis,23
207,minnesota,20
11,shooting,18
404,ice,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
229,2026-01-26-12-22-24 +0000,nypost,Powerful House Republican suggests Trump consi...,https://nypost.com/2026/01/26/us-news/powerful...,141
231,2026-01-26-12-03-00 +0000,wsj,Republican unity over President Trump’s immigr...,https://www.wsj.com/politics/policy/miller-noe...,121
208,2026-01-26-14-24-51 +0000,nypost,Trump sending border czar Tom Homan to Minneso...,https://nypost.com/2026/01/26/us-news/trump-se...,108
175,2026-01-26-16-07-05 +0000,cbc,Trump says he and Minnesota governor on 'simil...,https://www.cbc.ca/news/world/minnesota-court-...,105
193,2026-01-26-15-14-23 +0000,bbc,Trump says administration 'reviewing everythin...,https://www.bbc.com/news/articles/cr571qg4m61o...,104


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
229,141,2026-01-26-12-22-24 +0000,nypost,Powerful House Republican suggests Trump consi...,https://nypost.com/2026/01/26/us-news/powerful...
208,62,2026-01-26-14-24-51 +0000,nypost,Trump sending border czar Tom Homan to Minneso...,https://nypost.com/2026/01/26/us-news/trump-se...
170,56,2026-01-26-16-34-00 +0000,wsj,"Deadly Winter Storm Disrupts Air Travel, Leave...",https://www.wsj.com/us-news/climate-environmen...
272,50,2026-01-26-08-36-39 +0000,wsj,Stock Market Today: Dow Rises; Gold Price Tops...,https://www.wsj.com/livecoverage/stock-market-...
135,40,2026-01-26-18-26-00 +0000,wsj,Israel Says It Will Reopen Gaza Border Crossin...,https://www.wsj.com/world/middle-east/israel-t...
17,33,2026-01-26-22-35-24 +0000,nypost,Sicko Long Island cop caught in FBI sting pres...,https://nypost.com/2026/01/26/us-news/sicko-lo...
148,33,2026-01-26-17-54-24 +0000,nypost,At least 18 dead after ferry carrying over 350...,https://nypost.com/2026/01/26/world-news/at-le...
91,32,2026-01-26-20-32-47 +0000,latimes,A gunman is spotted near a grade school. O.C. ...,https://www.latimes.com/california/story/2026-...
27,28,2026-01-26-22-21-29 +0000,startribune,Judge hears arguments over whether federal gov...,https://www.startribune.com/judge-hears-argume...
268,28,2026-01-26-09-09-00 +0000,wsj,Nvidia-Backed AI Startup Synthesia Raises Fund...,https://www.wsj.com/business/entrepreneurship/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
